In [1]:
# Add any additional libraries or submodules below

# Display plots inline
%matplotlib inline

# Data libraries
import pandas as pd
import numpy as np

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting defaults
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['figure.dpi'] = 80

# sklearn modules
import sklearn

# packages for the different models
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, KBinsDiscretizer
from sklearn.model_selection import GridSearchCV, KFold, train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, RidgeCV, LogisticRegression
from sklearn.tree import DecisionTreeRegressor 
from sklearn.kernel_ridge import KernelRidge

### Data

In [2]:
sales = pd.read_csv("sales.csv")
sales_test = pd.read_csv("sales_test.csv")

### Preprocessing

In [3]:
def preprocessing(sales):
    """
    Takes in dataframe sales, does the preprocessing and outputs
    the featues we want to model as dataframe X.
    """
    # Bin the neighorhoods in groups of 6
    bin1 = ['nb_02', 'nb_09', 'nb_06', 'nb_13', 'nb_23', 'nb_01']
    bin2 = ['nb_11', 'nb_17', 'nb_18', 'nb_07', 'nb_20', 'nb_22']
    bin3 = ['nb_24', 'nb_04', 'nb_08', 'nb_14', 'nb_05', 'nb_16']
    bin4 = ['nb_21', 'nb_15', 'nb_10', 'nb_12', 'nb_03', 'nb_19']
    
    # Create 4 new features: neighborhood groups with an increasing mean
    sales['nbh_1'] = sales['neighborhood'].isin(bin1).astype(int)
    sales['nbh_2'] = sales['neighborhood'].isin(bin2).astype(int)
    sales['nbh_3'] = sales['neighborhood'].isin(bin3).astype(int)
    sales['nbh_4'] = sales['neighborhood'].isin(bin4).astype(int)

    # Combine year_sold and year_built into 1 column age
    sales['age'] = sales['year_sold'] - sales['year_built']
    
    # log transform sale_price and lot_area
    sales["log_sale_price"] = np.log(sales["sale_price"])
    sales["log_lot_area"] = np.log(sales["lot_area"])
    
    # Create dataframe X and drop all the features we do not want in our model
    X = sales.copy()
    X = X.drop(columns = ["sale_price", "log_sale_price", "garage_area", "year_sold", 
                          "year_built", "neighborhood", "lot_area", "full_bath"])
    
    # Transform the categorical variables into dummys,
    # to prevent rank deficiency we add drop_first = True
    X = pd.get_dummies(X, drop_first = True)
    
    return X

In [4]:
# training data
X = preprocessing(sales)
y = sales.log_sale_price

# test data
X_test = preprocessing(sales_test)
y_test = sales_test.log_sale_price

# Scaling the data
X_merged = pd.concat([X, X_test])
S = StandardScaler().fit(X_merged)

X_scaled = S.transform(X)
X_test_scaled = S.transform(X_test)

## Models

In [5]:
# Start with 2 helper functions
def model_fit(m, X, y, plot = False):
    """Returns the root mean squared error of a fitted model based on provided X and y values.
    
    Args:
        m: sklearn model object
        X: model matrix to use for prediction
        y: outcome vector to use to calculating rmse and residuals
        plot: boolean value, should fit plots be shown 
    """
    
    y_hat = m.predict(X)
    rmse = mean_squared_error(y, y_hat, squared=False)
    true_rmse = mean_squared_error(np.exp(y), np.exp(y_hat), squared=False)
    
    res = pd.DataFrame(
        data = {'y': y, 'y_hat': y_hat, 'resid': y - y_hat}
    )
    
    if plot:
        plt.figure(figsize=(12, 6))
        
        plt.subplot(121)
        sns.lineplot(x='x', y='y', color="grey", data =  pd.DataFrame(data={'x': [min(y),max(y)], 'y': [min(y),max(y)]}))
        sns.scatterplot(x='y', y='y_hat', data=res).set_title("Fit plot")
        
        plt.subplot(122)
        sns.scatterplot(x='y', y='resid', data=res).set_title("Residual plot")
        
        plt.subplots_adjust(left=0.0)
        
        plt.suptitle("Model rmse = " + str(round(rmse, 4)), fontsize=16)
        plt.show()
    
    return rmse, true_rmse

def get_coefs(m):
    """Returns the model coefficients from a Scikit-learn model object as an array,
    includes the intercept if available.
    """
    
    if m.intercept_ is None:
        return m.coef_
    
    return np.concatenate([[m.intercept_], m.coef_])

### Linear Regression

In [12]:
lr = LinearRegression().fit(X, y)

# model evaluation for trainig set
lr_rmse = model_fit(lr, X, y, plot=False)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(lr_rmse[0]))
print("\n")

# model evaluation for testing set
lr_test_rmse = model_fit(lr, X_test, y_test, plot=False)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(lr_test_rmse[0]))

The model performance for training set
--------------------------------------
RMSE is 0.11120661717953698


The model performance for testing set
--------------------------------------
RMSE is 0.11211512775305356


### Regression trees

In [7]:
dt = GridSearchCV(
     DecisionTreeRegressor(),
     param_grid={"max_depth": range(1, 10),
                 "min_samples_leaf": range(1, 10)},
     cv=KFold(5, True, random_state=1234),
    scoring="neg_root_mean_squared_error"
).fit(X, y)

# model evaluation for training set
dt_rmse = model_fit(dt, X, y, plot=False)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(dt_rmse[0]))
print('max_depth is', dt.best_params_['max_depth'])
print('min_samples_leaf', dt.best_params_['min_samples_leaf'])
print("\n")


# model evaluation for testing set
dt_test_rsme = model_fit(dt, X_test, y_test, plot=False)   

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(dt_test_rsme[0]))

The model performance for training set
--------------------------------------
RMSE is 0.08719255055352185
max_depth is 9
min_samples_leaf 2


The model performance for testing set
--------------------------------------
RMSE is 0.15788808218082026


### Ridge Regression

In [8]:
r_cv = RidgeCV(
    alphas = np.linspace(0.1, 20, num=200), # RidgeCV does not allow alpha=0 for some reason
    scoring = "neg_mean_squared_error"
).fit(X_scaled, y)

# model evaluation for training set
ridge_rmse = model_fit(r_cv, X_scaled, y, plot=False)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(ridge_rmse[0]))
print("The optimal alpha is", round(r_cv.alpha_, 1))
print("\n")

# model evaluation for testing set

ridge_test_rmse = model_fit(r_cv, X_test_scaled, y_test, plot = False)
    
print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(ridge_test_rmse[0]))

The model performance for training set
--------------------------------------
RMSE is 0.11124306072835116
The optimal alpha is 12.0


The model performance for testing set
--------------------------------------
RMSE is 0.11213073528659062


### Kernel Ridge Regression

In [9]:
kr = GridSearchCV(KernelRidge(kernel='rbf'),
                 param_grid={"alpha": np.logspace(-2, 0, num=4),
                             "gamma": np.logspace(0, 3, num=50)},
                 scoring = 'neg_mean_squared_error', cv=5)

kr = kr.fit(X, y)
rmse_kr = model_fit(kr, X, y, plot = False)

rmse_test_kr = model_fit(kr, X_test, y_test, plot = False)

print("The optimal alpha is", kr.best_params_["alpha"])
print("The optimal gamma is", kr.best_params_["gamma"])

The optimal alpha is 0.01
The optimal gamma is 1.0


In [10]:
# model evaluation for training set
print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse_kr[0]))
print("\n")

# model evaluation for testing set
print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse_test_kr[0]))

# overfitted

The model performance for training set
--------------------------------------
RMSE is 0.1165383663639526


The model performance for testing set
--------------------------------------
RMSE is 11.75296449421215


### Lasso Regression

In [11]:
def optimal_alpha(x='x', y='y'):
    alphas = np.linspace(0.01, 1, num=100)

    l_gs = GridSearchCV(
        Lasso(),
        param_grid={'alpha': alphas},
        cv=KFold(5, True, random_state=1234),
        scoring="neg_root_mean_squared_error"
    ).fit(x, y)

    l_cv_res = pd.DataFrame().assign(
        alpha = alphas,
        rmse = -1 * l_gs.cv_results_['mean_test_score'],           # mean of the rmse over the folds
        rmse_se = l_gs.cv_results_['std_test_score'] / np.sqrt(l_gs.n_splits_), # standard error of the rmse
    )

    i = l_cv_res.rmse.idxmin()

    min_rmse = l_cv_res.rmse[i]       # Smallest rmse
    min_rmse_se = l_cv_res.rmse_se[i] # Std error of the smallest rmse

    sub = l_cv_res.rmse <= min_rmse + min_rmse_se # Find rmses w/in 1se of the min + se

    alpha_opt = l_cv_res.alpha[ sub ].max() # Find the largest alpha
    return alpha_opt


# model evaluation for training set
alpha_opt = optimal_alpha(x=X_scaled, y=y)

lasso = Lasso(alpha_opt).fit(X_scaled, y)
rmse_lasso = model_fit(lasso, X_scaled, y, plot=False)

print("The model performance for training set")
print("--------------------------------------")
print("The optimal alpha is", alpha_opt)
print('RMSE is {}'.format(rmse_lasso[0]))
print("\n")

# model evaluation for testing set
lasso_test = Lasso(alpha_opt).fit(X_test_scaled, y_test)
rmse_test_lasso = model_fit(lasso_test, X_test_scaled, y_test, plot=False)
    
print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse_test_lasso[0]))

The model performance for training set
--------------------------------------
The optimal alpha is 0.01
RMSE is 0.11682673544441437


The model performance for testing set
--------------------------------------
RMSE is 0.1127330746826424
